In [1]:
#!pip install -r requirements.txt -q

In [3]:
import gmpy2
import numpy as np
from preprocessing import *
from sage.libs.pari.all import pari
from sage.all import ZZ, PolynomialRing, GF

SEED = 777
rng = np.random.default_rng(SEED)

pari.allocatemem(2 * 1024**3)

PARI stack size set to 2147483648 bytes, maximum size set to 2147483648


Generar el algebra $A_q$ dado el campo $p^k$ y el tamano del mensaje

In [4]:
n = 3
# 2**64 + 13
p = gmpy2.mpz(2)**16 + 1  # primo que define el cuerpo de base 𝔽_p
k = 1   # queremos incluir 𝔽_{p^k} dentro de A
# p = 2
# k = 8
s = 3   # número de copias de 𝔽_{p^k} que necesitamos

tau = p/2
sec = 40
delta = 1.0052

min_m = 14620
max_m = 50000  # rango de búsqueda de índices ciclotómicos m
m, phi_m, coeffs = calculate_m(p, k, s, min_m, max_m)

# m = 17425
# phi_m = int(euler_phi(m))

# Phi_ZZ = cyclotomic_polynomial(m)
# R = PolynomialRing(Zmod(p), 'x')
# Phi = R(Phi_ZZ)    
# coeffs = [int(c) for c in Phi.list()]
N = phi_m
# print("(m, N)=", (m, N))

c_sec = 9 * np.power(float(N), 2) * np.power(float(sec), 4) * np.power(2.0, sec + 8)


m = 32768 phi(m) = 16384 degrees: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [5]:
factors = unique_prime_factors(m)
if len(factors) == 1:
    C_m = 4/np.pi
    print("m es primo, asi que C_m =", C_m)
elif np.prod(factors) <= 400:
    C_m = 8.6
    print("los factores de m son pequeños, entonces C_m =", C_m)
else:
    # Ejecutar en paralelo (aumenta trials para mayor precisión)
    C_m = float(calculate_Cm_parallel(m, total_trials=1000))

    print(f"\nResultado Monte Carlo Paralelo:")
    print(f"C_m estimado: {C_m:.2f}")

    # Margen de seguridad (x1.5 o x2 es común en la práctica)
    margin = 1
    cm_seguro = C_m * margin

    print(f"Valor recomendado (con margen x{margin}): {cm_seguro:.2f}")

m es primo, asi que C_m = 1.2732395447351628


## 💡 El Problema Central: Dependencia Circular

Imagina que estás tratando de ajustar dos perillas, $q$ (tamaño del contenedor) y $r$ (cantidad de ruido), pero están conectadas por una cuerda elástica:
1. Regla de Corrección: Si subes el ruido ($r$), necesitas un contenedor ($q$) más grande para que los datos quepan.
    - Causa: A mayor $r \to$ mayor $q$
2. Regla de Seguridad: Si agrandas el contenedor ($q$), el problema matemático se vuelve "más fácil" de romper, por lo que necesitas subir el ruido ($r$) para esconder el secreto de nuevo.
    - Causa: A mayor $q \to$ mayor $r$.

Esto crea un bucle: Subir $r$ te obliga a subir $q$, lo cual te obliga a subir $r$ de nuevo...El algoritmo busca el punto exacto donde dejas de necesitar subir ambos valores. Ese es el "Punto Fijo".

In [6]:
# Valor inicial semilla para r (rho)
r_seed = 3.2 

# Ejecutar el método numérico
q, r = fixed_point_qr(r_seed, N, C_m, p, tau, c_sec, n, sec, delta)

print("\n=============================================")
print("      RESULTADOS DE PARÁMETROS OPTIMIZADOS")
print("=============================================")
print(f"1. Módulo (q)        : {int(q):.4e}")
print(f"   En potencias de 2 : 2^{gmpy2.log2(q):.2f}")
print("---------------------------------------------")
print(f"2. Ruido std (r/rho) : {r:.6f}")
print(f"   (Debe ser >= 3.2) : {'CUMPLE' if r >= 3.2 else 'FALLA'}")
print("=============================================")

--- Iniciando Iteración (N=16384, sec=40) ---
✅ Convergencia en iteración 1

      RESULTADOS DE PARÁMETROS OPTIMIZADOS
1. Módulo (q)        : 7.6957e+112
   En potencias de 2 : 2^375.00
---------------------------------------------
2. Ruido std (r/rho) : 3.200000
   (Debe ser >= 3.2) : CUMPLE


## Generate keys

In [7]:
Aq = generate_Aq(q, coeffs)

# Key Generation
a = Aq.random_element()
s_coeff, e_coeff= sample_discrete_gaussian_ZN(N, r, q, rng, num_samples=2)
sk = Aq(s_coeff.tolist())
e = Aq(e_coeff.tolist())
b = (a*sk) + (p*e)

pk = (a, b)
# sk

# Key hat Generation
a_hat = Aq.random_element()
b_hat = Aq.random_element()

pk_hat = (a_hat, b_hat)

Construido A_q = Univariate Quotient Polynomial Ring in a over Ring of integers modulo 76957043352332967211482500195592995713046365762627825523336510555167425334955489475418488779072100860950445293568 with modulus x^16384 + 1


## Encode/Decode messages

In [8]:
print("\n--- Iniciando Prueba de Encoding/Decoding ---")

# 1. Preparar la estructura SIMD
R_p, slots_moduli = prepare_slots(p, coeffs)
num_slots = len(slots_moduli)
print(f"Sistema configurado con {num_slots} slots disponibles.")

# 2. Definir mensajes de prueba
# Solo usaremos 4 valores, el resto serán ceros
plaintext = [ord("H"), ord("o"), ord("l"), ord("a")] 

print(f"Mensajes originales: {plaintext}")

# 3. Codificar (Encode)
plaintext_poly = encode(plaintext, R_p, slots_moduli, Aq)

print("Codificación exitosa. Elemento en Aq creado.")
# print(plaintext_poly) # Descomentar para ver el polinomio gigante

# 4. Decodificar (Decode) para verificar
decoded_message = decode(plaintext_poly, R_p, slots_moduli)

# Filtramos solo los primeros 4 para comparar visualmente
print(f"Mensajes recuperados: {decoded_message[:4]}")

# Verificación automática
assert decoded_message[:4] == plaintext
print("¡Éxito! Los mensajes coinciden.")


--- Iniciando Prueba de Encoding/Decoding ---
Sistema configurado con 16384 slots disponibles.
Mensajes originales: [72, 111, 108, 97]
Codificación exitosa. Elemento en Aq creado.
Mensajes recuperados: [72, 111, 108, 97]
¡Éxito! Los mensajes coinciden.


## Encrypt/Decrypt

In [9]:
# Encryption
ciphertext = encrypt(plaintext, N, r, q, rng, R_p, slots_moduli, Aq, p, pk)

# Decryption
decrypted_messages = decrypt(ciphertext, R_p, slots_moduli, sk)

# Verificación automática
assert decrypted_messages[:4] == plaintext
print("¡Éxito! Los mensajes coinciden.")

¡Éxito! Los mensajes coinciden.
